# DS18B20: Bring up and basic usage of the temperature sensor

*Important:*  
The **DS18B20** is a digital thermometer and provides 9-bit to 12-bit Celsius temperature measurements. It has an alarm function with nonvolatile user-programmable upper and lower trigger points. The DS18B20 communicates over a **1-Wire** bus that by definition requires only one data line (and ground) for communication with a central microprocessor.

- Data sheet: [https://datasheets.maximintegrated.com/en/ds/DS18B20.pdf](https://datasheets.maximintegrated.com/en/ds/DS18B20.pdf)
- Python **hidapi** library: [https://pypi.org/project/hidapi/}(https://pypi.org/project/hidapi/) <span style="color:green">[works]</span>
- Python **hid** library: [https://pypi.org/project/hid/](https://pypi.org/project/hid/) <span style="color:red">[does NOT work]</span>


## Installation of required development libraries

```
$ sudo apt install python-dev libusb-1.0-0-dev libudev-dev
```

## Installation of required Python libraries

The installation takes place in a Python environment:
```
$ source ~/jupyter-env/bin/activate
$ pip install hidapi
```

## Udev rule

To be able to address the device, you need to create a rule file in udev rules directory. Make sure that your current user belongs to the *plugdev* group.
```
$ sudo echo 'SUBSYSTEM=="usb", ATTRS{idVendor}=="16c0", ATTRS{idProduct}=="0480", MODE="0660", GROUP="plugdev"' > /etc/udev/rules.d/99-tempsensor.rules
```

Activate the new rule by calling
```
$ sudo udevadm control --reload-rules
```

## Sample usage code

Use the enumerate() function to return information about accessible HID devices on USB:

In [15]:
import hid

for device_dict in hid.enumerate():
    keys = list(device_dict.keys())
    keys.sort()
    for key in keys:
        print("%s : %s" % (key, device_dict[key]))
    print()

interface_number : 0
manufacturer_string : DIAMEX GmbH
path : b'1-1.4:1.0'
product_id : 1152
product_string : Temp-Sensor-Tester
release_number : 256
serial_number : 
usage : 0
usage_page : 0
vendor_id : 5824



The details about a HID device can be printed with following code:

In [20]:
vid = 0x16c0 # Change it for your device
pid = 0x0480 # Change it for your device

h = hid.device()
h.open(vid, pid)
#h.open_path(b'1-1.4:1.0')

print("Manufacturer: %s" % h.get_manufacturer_string())
print("Product: %s" % h.get_product_string())
print("Serial No: %s" % h.get_serial_number_string())

#h.close()

Manufacturer: DIAMEX GmbH
Product: Temp-Sensor-Tester
Serial No: Љ


In [38]:
connected_with = '{:s} ({:s}) with idVendor={:#06x} and idProduct={:#06x} over USB'.format(h.get_product_string(), h.get_manufacturer_string(), vid, pid)
connected_with

'Temp-Sensor-Tester (DIAMEX GmbH) with idVendor=0x16c0 and idProduct=0x0480 over USB'

In [56]:
import hid
import time

INTERVAL = 1.0
VID = 0x16c0 # vendor id of usb device
PID = 0x0480 # product id of usb device

try:
    h = hid.device()
    h.open(VID, PID)
    print("Device Info:")
    print("\t Manufacturer: %s" % h.get_manufacturer_string())
    print("\t Product: %s" % h.get_product_string())

except Exception as ex:
    print("Connecting with the device raised the error: '{}'".format(ex))
    raise

while True:
    try:
        d = h.read(64)

        if d:
            sensor_id = ''
            # get sensor ID with the bytes 8 to 16 from character buffer
            for i in range(8, 16, 1):
                # convert integer from byte list to string with padding
                hex_string = '{:02x}'.format(d[i])
                sensor_id = sensor_id + hex_string + ' '

            # combine high and low byte of temperature value and convert to float
            temp = float(d[5] << 8 | d[4]) / 10

            print("<{:s}> Sensor: {:d} of {:d}, Temperature: {:.2f} °C, ID: {}".format(time.strftime('%H:%M:%S'), d[1], d[0], temp, sensor_id))

        else:
            print("Could not read from device!")
            break
    
        time.sleep(INTERVAL)
        
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        h.close()
        break

Device Info:
	 Manufacturer: DIAMEX GmbH
	 Product: Temp-Sensor-Tester
<19:36:06> Sensor: 2 of 2, Temperature: 21.20 °C, ID: 28 84 28 e2 2c 20 01 10 
<19:36:07> Sensor: 1 of 2, Temperature: 21.50 °C, ID: 28 64 8e 75 d0 01 3c 81 
<19:36:08> Sensor: 2 of 2, Temperature: 21.20 °C, ID: 28 84 28 e2 2c 20 01 10 
Keyboard Interrupt ^C detected.
Bye.


## Using the wrapper class 'DS18B20_over_USB'

The new wrapper class **DS18B20_over_USB** in the python file *DS18B20_class.py* implements the communication with the the temperature sensor DS18B20 via USB interface board *Temp-Sensor-Tester* from *DIAMEX GmbH*. This interface board displays the connected DS18B20 temperature sensor(s) and their readings as a Human Interface Device (HID) device. The communication with the HID device is realized with the cython package *hidapi*.

In [1]:
# import wrapper class DS18B20_class from python file DS18B20_over_USB.py
from DS18B20_class import DS18B20_over_USB

In [2]:
# create new device object for the USB interface board
if_board = DS18B20_over_USB(vid = 0x16c0, pid = 0x0480)

In [3]:
# read connection state to the USB interface board
if_board.status

'Connected'

In [4]:
# read connection details
if_board.connected_with

'Temp-Sensor-Tester (DIAMEX GmbH) with idVendor=0x16c0 and idProduct=0x0480 over USB'

In [18]:
# close the connection to the USB interface board
if_board.closeConnection()

In [4]:
# open the connection again
if_board.openConnection(vid = 0x16c0, pid = 0x0480)

In [7]:
# get an array of sensor id(s)
if_board.getSensorIDs()

['28 64 8e 75 d0 01 3c 81', '28 84 28 e2 2c 20 01 10']

In [15]:
# read sensor values to dictionary
temp_dict = if_board.getTemperature_dict()
temp_dict

{'28 84 28 e2 2c 20 01 10': 21.7, '28 64 8e 75 d0 01 3c 81': 22.1}

In [16]:
temp_dict['28 84 28 e2 2c 20 01 10']

21.7

In [17]:
temp_dict['28 64 8e 75 d0 01 3c 81']

22.1

In [8]:
import time

# import wrapper class DS18B20_class from python file DS18B20_over_USB.py
from DS18B20_class import DS18B20_over_USB

INTERVAL = 1.0
VID = 0x16c0 # vendor id of usb device
PID = 0x0480 # product id of usb device

# create new device object for the USB interface board
if_board = DS18B20_over_USB(VID, PID)

# get an ordered array of sensor id(s)
sensor_ids_array = if_board.getSensorIDs()
sensor_ids_cnt = len(sensor_ids_array)

while True:
    try:
        temp_dict = if_board.getTemperature_dict()

        for i in range(0, sensor_ids_cnt, 1):
            sensor_id = sensor_ids_array[i]
            sensor_temp = temp_dict[sensor_id]

            print("<{:s}> Sensor: {:d} of {:d}, Temperature: {:.1f} °C, ID: {}".format(time.strftime('%H:%M:%S'), i+1, sensor_ids_cnt, sensor_temp, sensor_id))

        time.sleep(INTERVAL)
        
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        # close the connection to the USB interface board
        if_board.closeConnection()
        break

<22:00:01> Sensor: 1 of 2, Temperature: 22.10 °C, ID: 28 64 8e 75 d0 01 3c 81
<22:00:01> Sensor: 2 of 2, Temperature: 21.60 °C, ID: 28 84 28 e2 2c 20 01 10
<22:00:04> Sensor: 1 of 2, Temperature: 22.10 °C, ID: 28 64 8e 75 d0 01 3c 81
<22:00:04> Sensor: 2 of 2, Temperature: 21.60 °C, ID: 28 84 28 e2 2c 20 01 10
<22:00:07> Sensor: 1 of 2, Temperature: 22.10 °C, ID: 28 64 8e 75 d0 01 3c 81
<22:00:07> Sensor: 2 of 2, Temperature: 21.60 °C, ID: 28 84 28 e2 2c 20 01 10
<22:00:10> Sensor: 1 of 2, Temperature: 22.10 °C, ID: 28 64 8e 75 d0 01 3c 81
<22:00:10> Sensor: 2 of 2, Temperature: 21.60 °C, ID: 28 84 28 e2 2c 20 01 10
<22:00:13> Sensor: 1 of 2, Temperature: 22.10 °C, ID: 28 64 8e 75 d0 01 3c 81
<22:00:13> Sensor: 2 of 2, Temperature: 21.60 °C, ID: 28 84 28 e2 2c 20 01 10
<22:00:16> Sensor: 1 of 2, Temperature: 22.10 °C, ID: 28 64 8e 75 d0 01 3c 81
<22:00:16> Sensor: 2 of 2, Temperature: 21.60 °C, ID: 28 84 28 e2 2c 20 01 10
<22:00:19> Sensor: 1 of 2, Temperature: 22.10 °C, ID: 28 64 8e 7